In [33]:
#coding:utf-8
import cv2
import numpy as np
import glob
import os

#棋盘格模板规格
w, h = 9, 6
#世界坐标系中的棋盘格点,例如(0,0,0), (1,0,0), (2,0,0) ....,(8,5,0)，去掉Z坐标，记为二维矩阵
objp = np.zeros((w*h,3), np.float32)
objp[:,:2] = np.mgrid[0:w,0:h].T.reshape(-1, 2)
# 储存棋盘格角点的世界坐标和图像坐标对
objpoints = [] # 在世界坐标系中的三维点
imgpoints = [] # 在图像平面的二维点

images = glob.glob('camera_cal_pic/calibration*')
for fname in images:
    img = cv2.imread(fname)
    gray = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
    # 找到棋盘格角点
    ret, corners = cv2.findChessboardCorners(gray, (w,h))
    # 如果找到足够点对，将其存储起来
    if ret == True:
        objpoints.append(objp)
        #利用cv2.cornerSubPix可以更精细的查找角点坐标，如果查找到了，用这个，没查找到用cv2.findChessboardCorners中找到的角点
        corners2 = cv2.cornerSubPix(gray,corners,(11,11),(-1,-1),criteria = (cv2.TERM_CRITERIA_EPS + cv2.TERM_CRITERIA_MAX_ITER, 30, 0.001))
        if corners2.any():
                imgpoints.append(corners2)
        else:
                imgpoints.append(corners)
        # 将角点在图像上显示
        #cv2.drawChessboardCorners(img, (w,h), corners, ret)
        #cv2.imshow('findCorners',img)
        #cv2.waitKey(0)
    else:
        print('Warning: ret = %s for %s' % (ret, fname))
cv2.destroyAllWindows()

#利用图像中2d点和空间3d点计算旋转和平移矩阵，函数使用cv2.calibrateCamera，返回mtx（相机内参矩阵）, dist（畸变矩阵）
ret, mtx, dist, rvecs, tvecs = cv2.calibrateCamera(objpoints, imgpoints, gray.shape[::-1], None, None)

#图像去畸变
test_images = glob.glob("testImage/test*")
for per in test_images:
    test_image = cv2.imread(per)
    dir, name = os.path.split(per)
    dst = cv2.undistort(test_image, mtx, dist, None, None)
    cv2.imwrite('test_image_undistort/' + name, dst)

"\nimg2 = cv2.imread('./camera_cal_pic/calibration1.jpg')\ndst = cv2.undistort(img2, mtx, dist, None, None)\ncv2.imwrite('./camera_cal_pic_undistort/calibration1_undistort.jpg',dst)\n"